In [5]:
#what to install 
# sudo  apt-get install python3-bs4
!pip install yfinance


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [1]:
import pandas as pd
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import yfinance as yf
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
# step1 : python 3.10 pe kaam karna hai
import pandas as pd
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import yfinance as yf
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
#list of tickers
ticker_symbol = ['AAPL','GOOG','INTU','ALGN','AMZN','AAL','MSFT','AMAT','FOX',
          'SBUX','WDC','NFLX','COST','ADBE','EBAY','WDAY','LRCX','BIDU','PYPL']

# Create an empty dataframe to store the data
df = pd.DataFrame(columns=['Date', 'News Headline', 'Closing Price', 'Next Day Prediction'])


# Set the date range for the data collection
end_date = datetime.now()
start_date = end_date - timedelta(days=1500)
for ticker in ticker_symbol:
    print(ticker)
    url = f'https://news.google.com/rss/search?q={ticker}&hl=en-US&gl=US&ceid=US:en'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'xml')
    news_items = soup.findAll('item')
        

    # Collect the stock prices for the company of interest
    stock_data = yf.download(ticker, start=start_date, end=end_date)

 
    
        # Loop through the news items and calculate the sentiment score
    for news_item in news_items:
        news_date = datetime.strptime(news_item.pubDate.string, '%a, %d %b %Y %H:%M:%S %Z')
        if start_date <= news_date <= end_date:
            news_headline = news_item.title.string
            
            try:
                closing_price = stock_data.loc[str(news_date.date())]['Close']
                next_day_prediction = stock_data.loc[str((news_date + timedelta(days=1)).date())]['Close']
            except KeyError:
                # Skip this news item if stock data is not available
                continue
            df = df.append({'Date': news_date, 'News Headline': news_headline, 'News Body': news_body,
                             'Closing Price': closing_price, 'Next Day Prediction': next_day_prediction},
                        ignore_index=True)

    

In [33]:
# Save the data to a CSV file
df.to_csv('csv_files/all_tickers.csv', index=False)

In [40]:
df.drop('News Body', axis=1, inplace=True)
df.to_csv('csv_files/all_tickers.csv', index=False)

In [41]:
df


,Date,News Headline,Closing Price,Next Day Prediction
0,2023-04-27 14:02:02,Analysts Estimate Apple (AAPL) to Report a Dec...,168.410004,169.679993
1,2023-04-27 16:26:44,Apple's Q2 Preview: Why It May Be Painful (NAS...,168.410004,169.679993
2,2023-04-27 14:59:28,"Samsung, Apple lead weak smartphone market - S...",168.410004,169.679993
3,2023-04-27 21:56:01,Apple Pay Later won't affect us for years: Kla...,168.410004,169.679993
4,2023-04-26 15:59:46,13 Best Money Making Stocks to Buy Now - Yahoo...,163.759995,168.410004
...,...,...,...,...
1216,2022-10-06 07:00:00,Shareholder Alert: Robbins LLP Informs Shareho...,94.419998,90.169998
1217,2022-10-10 07:00:00,The one-year loss for PayPal Holdings (NASDAQ:...,84.519997,83.089996
1218,2022-12-14 08:00:00,"PYPL, SHOP, or ROKU: Which Growth Stock is Wor...",72.629997,69.769997
1219,2023-02-09 08:00:00,Markets Slide on Interest Rate Ruminations; PY...,78.419998,80.800003


## sort news according to date

In [42]:
df = df.sort_values('Date')
df

,Date,News Headline,Closing Price,Next Day Prediction
272,2020-10-22 07:00:00,Align Technology (ALGN) Stock Surges To New Hi...,453.230011,469.559998
260,2021-01-07 10:45:59,"Align Tech Stock (NASDAQ:ALGN), Quotes and New...",558.359985,570.530029
1152,2021-02-10 08:00:00,Should You Buy Baidu Inc. (BIDU) in 2021? - Ya...,304.049988,309.589996
199,2022-01-24 08:00:00,Intuit (INTU) stock forecast: Will the rally c...,534.679993,515.989990
1066,2022-01-27 08:00:00,Lam Research (LRCX) Q4 2021 Earnings Call Tran...,555.299988,561.140015
...,...,...,...,...
3,2023-04-27 21:56:01,Apple Pay Later won't affect us for years: Kla...,168.410004,169.679993
1204,2023-04-27 21:56:01,Apple Pay Later won't affect us for years: Kla...,74.269997,76.000000
954,2023-04-27 22:00:33,"Minnesota state representative: ""There is no p...",187.130005,186.139999
747,2023-04-27 22:48:00,Canadian Senate passes bill to compel local co...,325.850006,329.929993


In [44]:
# create new column 'stock' with values 1 or 0 based on the condition
df['stock'] = df.apply(lambda x: 1 if x['Next Day Prediction'] > x['Closing Price'] else 0, axis=1)

# re-arrange the columns as per your preference
df_new = df[['Date', 'News Headline', 'stock']]
df_new = df_new.reset_index(drop=True)
df_new

,Date,News Headline,stock
0,2020-10-22 07:00:00,Align Technology (ALGN) Stock Surges To New Hi...,1
1,2021-01-07 10:45:59,"Align Tech Stock (NASDAQ:ALGN), Quotes and New...",1
2,2021-02-10 08:00:00,Should You Buy Baidu Inc. (BIDU) in 2021? - Ya...,1
3,2022-01-24 08:00:00,Intuit (INTU) stock forecast: Will the rally c...,0
4,2022-01-27 08:00:00,Lam Research (LRCX) Q4 2021 Earnings Call Tran...,1
...,...,...,...
1216,2023-04-27 21:56:01,Apple Pay Later won't affect us for years: Kla...,1
1217,2023-04-27 21:56:01,Apple Pay Later won't affect us for years: Kla...,1
1218,2023-04-27 22:00:33,"Minnesota state representative: ""There is no p...",0
1219,2023-04-27 22:48:00,Canadian Senate passes bill to compel local co...,1


#
.
.
.
.
.
.
.
.
.
.
.
.





nothing jut making space

In [20]:
import pandas as pd

# Assume that `df` is the DataFrame you want to display as a table
table_html = df.to_html()

# Save the HTML table to a file
with open('table.html', 'w') as f:
    f.write(table_html)
    
from IPython.display import display

# Assume that `df` is the DataFrame you want to display as a table
display(df)


,Date,News Headline,News Body,Sentiment Score,Closing Price,Next Day Prediction
0,2023-04-27 14:02:02,Analysts Estimate Apple (AAPL) to Report a Dec...,"<a href=""https://news.google.com/rss/articles/...",0.0000,168.410004,169.679993
1,2023-04-27 16:26:44,Apple's Q2 Preview: Why It May Be Painful (NAS...,"<a href=""https://news.google.com/rss/articles/...",-0.4404,168.410004,169.679993
2,2023-04-27 14:59:28,"Samsung, Apple lead weak smartphone market - S...","<a href=""https://news.google.com/rss/articles/...",-0.4404,168.410004,169.679993
3,2023-04-27 21:56:01,Apple Pay Later won't affect us for years: Kla...,"<a href=""https://news.google.com/rss/articles/...",-0.1027,168.410004,169.679993
4,2023-04-26 15:59:46,13 Best Money Making Stocks to Buy Now - Yahoo...,"<a href=""https://news.google.com/rss/articles/...",0.6369,163.759995,168.410004
...,...,...,...,...,...,...
60,2023-02-06 08:00:00,A Rare Earnings Miss Couldn’t Stop Apple Stock...,"<a href=""https://news.google.com/rss/articles/...",-0.4215,151.729996,154.649994
61,2023-02-15 08:00:00,Apple (NASDAQ:AAPL) Stock: Legal Threat Looms ...,"<a href=""https://news.google.com/rss/articles/...",-0.4404,155.330002,153.710007
62,2023-01-17 08:00:00,Should Investors Bite Into Apple (AAPL) Stock?...,"<a href=""https://news.google.com/rss/articles/...",0.0000,135.940002,135.210007
63,2023-03-22 07:00:00,"AMZN, AAPL, or NFLX: Which FAANG Stock is the ...","<a href=""https://news.google.com/rss/articles/...",0.6767,157.830002,158.929993


In [89]:
# create new column 'stock' with values 1 or 0 based on the condition
df['stock'] = df.apply(lambda x: 1 if x['Next Day Prediction'] > x['Closing Price'] else 0, axis=1)

# re-arrange the columns as per your preference
df_new = df[['News Headline', 'Sentiment Score','stock']]
df_new

,News Headline,Sentiment Score,stock
0,Apple Inc.'s (NASDAQ:AAPL) institutional share...,0.8625,0
1,"JP Morgan hikes AAPL price target to $190, sti...",0.4404,0
2,Multiple Reasons to Invest in and Hold Apple (...,0.0000,0
3,Analysts are Upbeat About Apple’s (NASDAQ:AAPL...,0.0000,1
4,"In certain areas, India could be the new China...",0.2732,0
...,...,...,...
66,Is Apple Inc. (NASDAQ:AAPL) Worth US$153 Based...,0.3094,1
67,Should Investors Bite Into Apple (AAPL) Stock?...,0.0000,0
68,Will Apple (AAPL) Stock Bounce Back in 2023? -...,0.0000,1
69,Apple Stock Forecast: Could AAPL drop 19% from...,-0.3527,1


# save to csv

In [46]:
# Write data to CSV file
df_new.to_csv("csv_files/ddl_news.csv", index=False)

# New code

In [6]:
import yfinance as yf
import pandas as pd

# dates
start_date = "2020-01-01"
end_date = "2021-12-29"

# Download historical stock prices for Apple
ticker = yf.Ticker("AAPL")
data = ticker.history(start=start_date, end=end_date)

# Calculate next day's closing price and add as a new column
data["NextDayClose"] = data["Close"].shift(-1)

# Reformat data into desired columns
data = data.reset_index()
data = data[["Date", "Close", "NextDayClose"]]
data.columns = ["date", "closingprice", "nextdayPrice"]

# Write data to CSV file
data.to_csv("csv_files/apple_stock_prices.csv", index=False)

In [7]:
import pandas as pd

# Assume that `df` is the DataFrame you want to display as a table
table_html = data.to_html()

# Save the HTML table to a file
with open('table.html', 'w') as f:
    f.write(table_html)
    
from IPython.display import display

print("APPLE")
# Assume that `df` is the DataFrame you want to display as a table
display(data)

APPLE


,date,closingprice,nextdayPrice
0,2020-01-02 00:00:00-05:00,73.449394,72.735313
1,2020-01-03 00:00:00-05:00,72.735313,73.314911
2,2020-01-06 00:00:00-05:00,73.314911,72.970093
3,2020-01-07 00:00:00-05:00,72.970093,74.143898
4,2020-01-08 00:00:00-05:00,74.143898,75.718773
...,...,...,...
497,2021-12-21 00:00:00-05:00,171.729584,174.360291
498,2021-12-22 00:00:00-05:00,174.360291,174.995621
499,2021-12-23 00:00:00-05:00,174.995621,179.016129
500,2021-12-27 00:00:00-05:00,179.016129,177.983704


# google news 

### points
Internally, GoogleNews uses the beautifulsoup4 and requests libraries to parse and fetch news articles from Google News. So,we don't need to use beautifulsoup4 directly in our code.

In [9]:
pip install requests bs4 google-search-results
# pip install nytimesarticle


SyntaxError: invalid syntax (4280384247.py, line 1)

In [6]:
pip install nytimesarticle


Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
googlenews.enableException(True)

NameError: name 'googlenews' is not defined

In [10]:
from datetime import datetime, timedelta
import pandas as pd
from GoogleNews import GoogleNews

# Define start and end dates
start_date = datetime.strptime("2020-01-01", "%Y-%m-%d").date()
end_date = datetime.strptime("2021-04-10", "%Y-%m-%d").date()

# Initialize GoogleNews object
googlenews = GoogleNews(lang='en')

# Initialize empty list to store news articles
articles = []

DAILY_PRESENT = 0
NOT_PRESENT = 0
# Loop through each date between start and end date
current_date = start_date
while current_date <= end_date:


     # Set time range for current date
    from_date = current_date.strftime('%m/%d/%Y')
    to_date = (current_date + timedelta(days=1)).strftime('%m/%d/%Y')
    print(from_date)
    print(to_date)
    googlenews.set_time_range(from_date, to_date)

    # Get news articles for current date
    googlenews.search('AAPLE')
    news_articles = googlenews.result()
    
    # If there are no news articles for the current date, randomly select five articles about Apple from the last 30 days
    if len(news_articles) == 0:
        NOT_PRESENT += 1
        googlenews.set_period('7d')
        googlenews.search('AAPLE')
        if len(news_articles) > 5:
            news_articles = news_articles[:5]
    else:
        DAILY_PRESENT += 1
    
    # Add news articles to list
    for article in news_articles:
        article['date'] = current_date
    articles += news_articles
    
    # Move to next date
    current_date += timedelta(days=1)
    
    # Wait for a second to avoid overloading the server
    time.sleep(5)

# Convert list of news articles to pandas DataFrame
df_news = pd.DataFrame(articles, columns=['date', 'title', 'desc'])

# Save DataFrame to CSV file
df_news.to_csv('csv_files/apple_news.csv', index=False)


01/01/2020
01/02/2020
'NoneType' object is not iterable
'NoneType' object is not iterable


NameError: name 'time' is not defined

In [35]:
print("daily ",DAILY_PRESENT)
print("not_daily",NOT_PRESENT)

daily  0
not_daily 15


In [36]:
df_news

,date,title,desc
0,2020-01-10,Possible Spyware On Samsung Phones,I'm more inclined to think this is a biased ar...
1,2020-01-10,Possible Spyware On Samsung Phones,I'm more inclined to think this is a biased ar...
2,2020-01-10,Possible Spyware On Samsung Phones,I'm more inclined to think this is a biased ar...
3,2020-01-10,Possible Spyware On Samsung Phones,I'm more inclined to think this is a biased ar...
4,2020-01-10,Possible Spyware On Samsung Phones,I'm more inclined to think this is a biased ar...


In [12]:
'''
Scrape Google News using bs4
'''
from datetime import datetime, timedelta
from requests import get
from contextlib import closing
from bs4 import BeautifulSoup
import requests
from lxml import html
import pandas as pd
import time
# import datetime as datetime
import random
# Define start and end dates
# Define start and end dates
from datetime import datetime, timedelta

start_date = datetime.strptime("2023-04-01", "%Y-%m-%d").date()
end_date = datetime.strptime("2023-01-10", "%Y-%m-%d").date()

Ticker = 'Apple'
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'}
to_merge = pd.DataFrame({'Date': [],'Ticker': [], 'Url':[], 'headline': [], 'source': [], 'snippet': []})
it = 0

while start_date <= end_date:
    it += 1
    time.sleep(abs(random.gauss(0,4)))
    Date = str(start_date).replace("-","")
    URL = 'https://www.google.com/search?q='+Ticker+'+'+Date+'&source=lnms&tbm=nws'

    # Get URL
    raw_html = requests.get(URL, headers=headers)
    soup = BeautifulSoup(raw_html.text, 'lxml')
    title_list = soup.find_all("a", class_="l lLrAF")
    date_list = soup.find_all("span", class_="f nsa fwzPFf")
    source_list = soup.find_all("span", class_="xQ82C e8fRJf")
    snippet_list = soup.find_all("div", class_="st")

    # Parse elements
    title = []
    date = []
    source = []
    snippet = []
    url = []
    for elem in title_list:
        valid = str(elem).replace("<em>", "").replace("</em>", "")[19:-4]
        out1 = valid.split("=")[1][1:-6]
        out2 = valid.split("=")[-1].split('>')[-1]
        url.append(out1)
        title.append(out2)
    for elem in date_list:
        valid = str(elem)[27:-7]
        date.append(valid)
    for elem in source_list:
        valid = str(elem)[27:-7]
        source.append(valid)
    for elem in snippet_list:
        valid = str(elem).replace("<em>", "").replace("</em>", "")[16:-10]
        snippet.append(valid)
    if len(title) == len(date) and len(title) == len(url) and len(title) == len(source) and len(title) == len(snippet):
        ticker = ['AAPL']*len(title)
        news = pd.DataFrame({'Date':date,'Ticker': ticker,'Url': url, 'headline':title, 'source': source, 'snippet': snippet})
        to_merge = pd.concat([to_merge,news])
    
    # Move to next date
    start_date += timedelta(days=1)
    print(start_date)

to_merge.to_csv('csv_files/PYPL_news.csv')


In [13]:
import pandas as pd

In [14]:
import pandas as pd
from datetime import datetime, timedelta

# Set start and end dates
end_date = datetime.today()
start_date = end_date - timedelta(days=4*365)

# Create date range
date_range = pd.date_range(start=start_date, end=end_date, freq='D')

# Create dataframe with date and technical indicator data
data = {'Date': date_range,
        'SMA': [i*10 for i in range(len(date_range))],
        'RSI': [50 + i*2 for i in range(len(date_range))]}
df = pd.DataFrame(data)

# Save dataframe to CSV file
df.to_csv('csv_files/techindicators_20.csv', index=False)


In [4]:
import pandas as pd

In [6]:
import http.client
import json
import pandas as pd
conn = http.client.HTTPSConnection("google-news-api1.p.rapidapi.com")

headers = {
    'content-type': "application/octet-stream",
    'X-RapidAPI-Key': "121867993amshce30cba525c91ccp16eff1jsn296207f4e4b5",
    'X-RapidAPI-Host': "google-news-api1.p.rapidapi.com"
}

conn.request("GET", "/search?language=EN&q=APPLE&from=2023-01-01T00%3A00%3A01&to=2023-01-10T00%3A00%3A01&sort=date%3Aasc&limit=5", headers=headers)

res = conn.getresponse()
data = res.read()

# Parse the JSON response into a python dictionary
parsed_data = json.loads(data)

# Create a pandas dataframe using the dictionary
df = pd.DataFrame(parsed_data['articles'])
df

KeyError: 'articles'

In [ ]:
import http.client
import json
import pandas as pd

conn = http.client.HTTPSConnection("google-news-api1.p.rapidapi.com")

headers = {
    'content-type': "application/octet-stream",
    'X-RapidAPI-Key': "121867993amshce30cba525c91ccp16eff1jsn296207f4e4b5",
    'X-RapidAPI-Host': "google-news-api1.p.rapidapi.com"
}

conn.request("GET", "/search?language=EN&q=APPLE&from=2023-01-01T00%3A00%3A01&to=2023-01-10T00%3A00%3A01&sort=date%3Aasc&limit=5", headers=headers)

res = conn.getresponse()
data = res.read()

# Parse the JSON response into a python dictionary
parsed_data = json.loads(data)

# Save the data into a CSV file
with open('data.csv', 'w') as f:
    for item in parsed_data['articles']:
        f.write("%s\n" % item)

# Load the CSV file into a pandas DataFrame
df = pd.read_csv('data.csv', delimiter=',', header=None, names=['title', 'link', 'description', 'source', 'image'])

# Print the DataFrame
print(df)
